In [1]:
import numpy as np

In [2]:
import sys, os

In [3]:
sys.path.append(os.path.realpath('./../code/'))

In [114]:
import tools as tl
import numpy as np
import kwant
import kwant.continuum
from finite_system import finite_system


def lead_hamiltonian(mu_wire):
    h = (
        """
    ( t * (k_x**2 + k_y**2) - """
        + mu_wire
        + """ )* kron(sigma_0, sigma_z)
    + alpha * k_x * kron(sigma_y, sigma_z)
    - alpha * k_y * kron(sigma_x, sigma_z)
    + Delta_re * kron(sigma_0, sigma_x)
    + Delta_im * kron(sigma_0, sigma_y)
    + B_x * kron(sigma_y, sigma_0)
    """
    )
    return h


def infinite_system(**geometry):

    a = geometry["a"]
    w = geometry["w"]
    l = geometry["l"]
    
    center = geometry["center"]

    def wires(mu=1 > 0):
        """
        Function that span the wires region with mu.
        """

        def wires_shape(x, y):

        
            x0, y0 = center
            if -w / 2 - x0 < x < w / 2 - x0:
                if y0 - l <= y <= y0:
                    return mu
            return 0

        return wires_shape

    def make_leads():

        lead = kwant.Builder(symmetry=kwant.TranslationalSymmetry([0, a]))
        

        template_lead = kwant.continuum.discretize(
            lead_hamiltonian("mus_nw"), grid=a
        )

        
        def rectangle(site):
            x, y = site.pos
            if wires()(x, y):
                return True

        lead.fill(template_lead, shape=rectangle, start=[0, 0])

        lead.fill(
            template_lead,
            shape=lambda site: -w / 2 <= site.pos[0] < w / 2,
            start=[0],
        )


        return lead

    def f_params(**params):
        
        phi1 = params.pop("phi1")
        phi2 = params.pop("phi2")
        Delta = params.pop("Delta")
        params.update(Delta_re=1.0)
        params.update(Delta_im=0.0)

        params.update(mus_nw=params["mus_nw"][0])
    
        return params

    leads = make_leads()

    return leads.finalized(), f_params


In [115]:
import parameters

In [116]:
mu = parameters.bands[0]
params = parameters.junction_parameters(m_nw=[mu, mu, mu])

In [117]:
params

{'mus_nw': [0.0023960204649275973,
  0.0023960204649275973,
  0.0023960204649275973],
 't': 1.6561134509633838e-18,
 'Delta': 0.0005,
 'alpha': 3e-11,
 'B_x': 0.001,
 'phi1': 0,
 'phi2': 0,
 'sigma': 0,
 'a': 1e-08}

In [118]:
geometry = {
    "a": 1e-8,
    "w": 7 * 1e-8,
    'l': 150 * 1e-8, 
    "center": [0, 0]
}

In [119]:
lead_fn, f_params = infinite_system(**geometry)

/tmp/ipykernel_6347/2186061937.py:65: RuntimeWarning: fill(): The target builder already contains all starting sites.
  lead.fill(


In [120]:
import scipy

In [121]:
from utils import eigsh

In [133]:
def spectrum(syst, f_params, params, k=20):
    """Compute the k smallest magnitude eigenvalues and
    corresponding eigenvectors of a system.

    Parameters
    ----------
    syst : `kwant.FiniteSystem` or `kwant.InfiniteSystem`
        System of which the spectrum will be computed.
    params : dict
        Dictionary containing the parameters.
    k : int
        Number of eigenvalues to calculate.

    Returns
    -------
    energies : numpy array
        Array containing smallest magnitude eigenvalues.
    wfs : numpy array
        Array containing eigenvectors corresponding to eigenvalues.
    """
    ham = syst.hamiltonian_submatrix(params=f_params(**params), sparse=True)
    energies = eigsh(ham, k=k, sigma=0)
    return energies


def gap_from_band_structure(lead, f_params, params, Ns=31, full_output=False):
    def energies(k):
        k_x, k_y = k
        Es = spectrum(lead, f_params, dict(params, k_x=float(k_x), k_y=float(k_y)), k=4)[0]
        return np.abs(Es).min()
    
    result = scipy.optimize.brute(energies, ranges=((0, np.pi), (0, np.pi)),
                 Ns=Ns, full_output=full_output)

    return energies(result)

In [134]:
%%time

topo_gap = gap_from_band_structure(lead_fn, f_params, params)

CPU times: user 45.9 s, sys: 977 ms, total: 46.9 s
Wall time: 46.9 s


In [138]:
topo_gap

0.9995044745716134